In [19]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image

from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

In [11]:
train_data_generator = image.ImageDataGenerator(
    rescale= 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

train_img = train_data_generator.flow_from_directory(
    './training_set/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

test_data_generator = image.ImageDataGenerator(
    rescale=1./255,
)

test_img = test_data_generator.flow_from_directory(
    './test_set/test_set/',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)


val_data_generator = image.ImageDataGenerator(
    rescale=1./255,
)

val_img = test_data_generator.flow_from_directory(
    './val_images/',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.
Found 13 images belonging to 2 classes.


In [12]:
vgg_model = VGG16(include_top=False, input_shape=(64, 64, 3))

In [13]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [17]:
model = Sequential()

model.add(vgg_model)

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.Adam(learning_rate=0.00001),
    metrics=['accuracy']
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 256)               524544    
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 15,239,489
Trainable params: 15,239,489
Non-trainable params: 0
_________________________________________________________________


In [ ]:
es = EarlyStopping(monitor='loss', patience=5)

model.fit(
    train_img,
    epochs=20,
    callbacks=[es]
)

In [ ]:
model.save("best_vgg16_model")